#Pinecone Vector Store - Metadata Filter

In [4]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Build a Pinecone Index and connect to it

In [5]:
import pinecone

api_key = os.environ['PINECONE_API_KEY']
pinecone.init(api_key=api_key, environment="us-west1-gcp-free")

In [9]:
# dimensions are for text-embedding-ada-002
pinecone_index_name = 'debrief'
try:
  pinecone.create_index(pinecone_index_name, dimension=1536, metric="euclidean")
except Exception as e:
  print(f"Exception when creating pinecone index: {e}")

Exception when creating pinecone index: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Thu, 29 Jun 2023 22:36:05 GMT', 'x-envoy-upstream-service-time': '1524', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 1 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.



Build the PineconeVectorStore and VectorStoreIndex

In [10]:
pinecone_index = pinecone.Index(pinecone_index_name)

In [11]:
from llama_index import VectorStoreIndex, StorageContext
from llama_index.vector_stores import PineconeVectorStore

In [12]:
from llama_index.schema import TextNode

nodes = [
    TextNode(text='The Shawshank Redemption', metadata={
        "author": "Stephen King",
        "theme": "Friendship",
    }),
    TextNode(text='The Godfather', metadata={
        "director": "Francis Ford Coppola",
        "theme": "Mafia",
    }),
    TextNode(text="Inception", metadata={
        "director": "Christopher Nolan",
    })
]

In [13]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index, namespace='test_05_14')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes, storage_context=storage_context)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Upserted vectors: 100%|██████████| 3/3 [00:00<00:00, 10.99it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 39 tokens
> [build_index_from_nodes] Total embedding token usage: 39 tokens


Define metadata filters

In [14]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters
filters = MetadataFilters(filters=[ExactMatchFilter(key='theme', value='Mafia')])

No filters

In [17]:
retriever = index.as_retriever()
retriever.retrieve('What is inception about?')

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
> [retrieve] Total embedding token usage: 5 tokens


[NodeWithScore(node=TextNode(id_='25d88403-53e7-4438-998a-847ec4fa9ac4', embedding=None, metadata={'director': 'Christopher Nolan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9792a1fd7d2e1a08f1b1d70a597357bb6b68d69ed5685117eaa37ac9e9a3565e', text='Inception', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.856885135),
 NodeWithScore(node=TextNode(id_='8847b313-520b-44ac-b347-e475ec57258e', embedding=None, metadata={'director': 'Francis Ford Coppola', 'theme': 'Mafia'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ad2a08d4364262546db9711b915348d43e0ccc41bd8c3c41775e133624e1fa1b', text='The Godfather', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.768146574)]


Retrieve from vector store with filters

In [15]:
retriever = index.as_retriever(filters=filters)
retriever.retrieve('What is inception about?')

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
> [retrieve] Total embedding token usage: 5 tokens


[NodeWithScore(node=TextNode(id_='8847b313-520b-44ac-b347-e475ec57258e', embedding=None, metadata={'director': 'Francis Ford Coppola', 'theme': 'Mafia'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ad2a08d4364262546db9711b915348d43e0ccc41bd8c3c41775e133624e1fa1b', text='The Godfather', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.768146574)]


Use keyword arguments specific to pinecone

In [16]:
retriever = index.as_retriever(vector_store_kwargs={"filter": {"theme": "Mafia"}})
retriever.retrieve('What is inception about?')

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
> [retrieve] Total embedding token usage: 5 tokens


[NodeWithScore(node=TextNode(id_='8847b313-520b-44ac-b347-e475ec57258e', embedding=None, metadata={'director': 'Francis Ford Coppola', 'theme': 'Mafia'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ad2a08d4364262546db9711b915348d43e0ccc41bd8c3c41775e133624e1fa1b', text='The Godfather', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.768146574)]